In [1]:
import pandas as pd
import numpy as np
import torch
import pickle

from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.model_selection import train_test_split

2023-04-12 18:20:07.541192: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-12 18:20:21.534886: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-12 18:20:48.007193: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /spack/spack-0.17.1/apps/linux-rhel8-cascadelake/gcc-10.2.0/python-3.9.10-y63csltfuw5dh

In [2]:
def overlap_sequence(seq, word_length, overlap):
    if overlap >= word_length:
        print('Overlap must be less than word length')
        return
    
    for i in range(0, len(seq)-overlap, word_length-overlap):
        yield seq[i:i+word_length]
        
def get_overlap_array(seq, word_length=4, overlap=1):
    return np.array(list(overlap_sequence(seq, word_length, overlap)))

def get_overlap_string(seq, word_length=4, overlap=1):
    return ' '.join(list(overlap_sequence(seq, word_length, overlap)))

def compute_metrics(epred):
    # Computes metrics from specialized output from huggingface

    preds = np.exp(epred[0]) / np.sum(np.exp(epred[0]), axis = 0)
    labels = epred[1]

    metrics = {}
    metrics['auprc'] = average_precision_score(labels, preds[:,1])
    metrics['auroc'] = roc_auc_score(labels, preds[:,1])

    return metrics

In [7]:
PATH='/lustre/isaac/proj/UTK0196/deep-surface-protein-data/'
df = pd.read_csv(PATH+'M0059E_training_set.tsv', delimiter=',', header=0)

In [8]:
split_no = 1
inds = pickle.load(open('./splits/splits.pkl', 'rb'))
RUN = 0

In [9]:
train_inds, test_inds = inds[split_no][RUN]

In [11]:
#train_set = df.iloc[train_inds,:]
test_set = df.iloc[test_inds,:]
    
#train_set, val_set = train_test_split(train_set, test_size=0.1, random_state=1234)
    
#train_seqs_list = train_set['surf.sequence'].tolist() + train_set['deep.sequence'].tolist()
#train_seqs_labels = np.concatenate([np.zeros(train_set.shape[0], dtype=int), np.ones(train_set.shape[0], dtype=int)])
    
#val_seqs_list = val_set['surf.sequence'].tolist() + val_set['deep.sequence'].tolist()
#val_seqs_labels = np.concatenate([np.zeros(val_set.shape[0], dtype=int), np.ones(val_set.shape[0], dtype=int)])
    
test_seqs_list = test_set['surf.sequence'].tolist() + test_set['deep.sequence'].tolist()
test_seqs_labels = np.concatenate([np.zeros(test_set.shape[0], dtype=int), np.ones(test_set.shape[0], dtype=int)])
    
#classification_df_train = pd.DataFrame({'text' : train_seqs_list, 'label' : train_seqs_labels})
#classification_df_val = pd.DataFrame({'text' : val_seqs_list, 'label' : val_seqs_labels})
classification_df_test = pd.DataFrame({'text' : test_seqs_list, 'label' : test_seqs_labels})

#classification_df_train['text'] = classification_df_train['text'].transform(get_overlap_string)
#classification_df_val['text'] = classification_df_val['text'].transform(get_overlap_string)
classification_df_test['text'] = classification_df_test['text'].transform(get_overlap_string)
med_len = int(np.median([len(elem) for elem in classification_df_test['text']]))

#ds_train = Dataset.from_pandas(classification_df_train)
#ds_val = Dataset.from_pandas(classification_df_val)
ds_test = Dataset.from_pandas(classification_df_test)

In [32]:
tokenizer = AutoTokenizer.from_pretrained('tokenizers/AA-overlap-4_1', model_max_length=med_len, padding_side='left', truncation_side='right', return_token_type_ids=False)

#tokenized_ds_train = ds_train.map(lambda d : tokenizer(d['text'], truncation=True, padding=True), batched=True)
#tokenized_ds_val = ds_val.map(lambda d : tokenizer(d['text'], truncation=True, padding=True), batched=True)
tokenized_ds_test = ds_test.map(lambda d : tokenizer(d['text'], truncation=True, padding=True), batched=True)

Map:   0%|          | 0/92184 [00:00<?, ? examples/s]

In [35]:
model = AutoModelForSequenceClassification.from_pretrained('./models/custom-model-overlap-4_1_0/checkpoint-103500', num_labels=2)

In [34]:
out = model.predict(ds_test)
scores = compute_metrics(out)
print(scores)

AttributeError: 'DistilBertForSequenceClassification' object has no attribute 'predict'

In [31]:
uses_token_type_ids = ("token_type_ids" in model.__call__.__code__.co_varnames)
print(uses_token_type_ids)

False


In [37]:
ds_test['text']

['MMTN NTVS SEMN NLTS SYIW WTTI IWLS STII IPFS SVAT TSFC CSAE ENAV VIKK KNQR RPLL LGVQ QRWD DMYS SGKG GATQ QRQE ELGY YLPG GKQG GFLK KDPE EWHE ERAP PFFC CRLT TADV VDWV VEHP PAEA AGPL LWFN NYPF FSQ',
 'MAQT TDTK KRSG GDVA AGLT TNRS SMPE ESLA AAEA AAVL LGSM MIID DPEC CIGE EVVE ENLN NREA AFYR RSEH HQMI IFDA AVIA ALYE ERHR RGEA AIDG GVLL LRDE ELEK KRKQ QLEE EVGG GVEY YIAK KILD DSVP PSSA ANVT TYYS SGIV VKDK KQLL LRET TIAA AASE EILA ADAF FDES SGET TSEK KLDE EAER RRIF FAVT TDKR RISG GNAV VPLK KDLI ITRA AYEL LIEK KRDG GKLV VTVL LATG GYHE ELDD DKTC CGLQ QNGE EMII IIAG GRPS SMGK KTSI IALN NMAE EHIG GGID DKLP PVAI IFSL LEMG GRQQ QLAE ERFL LCSM MGEI IDAQ QKVR RKGM MLST TEDY YEKL LQRT TCAE ELYE EAPI IYVD DDTS STLT TPLE ELRA AKCR RRLK KSLY YDIR RCII IVDY YLQL LMHL LGSG GRVE ESRQ QQEI ITTI ISRY YLKA ALAR RELN NIPV VVVL LSQL LNRS SPEG GREG GHTP PRMS SDLR RESG GIIE EQDA ADVV VVLL LHRD DDYY YHRG GDKE EHEP PDNK KADL LIIA AKQR RNGP PTGS SVEP PIFR RERF FTRF FENA ASYA AEEP PAMR RLES SASG GAEE ETPF',
 'EMIV V

In [38]:
from transformers import pipeline
pipe = pipeline('text-classification', model=model, tokenizer=tokenizer)
pipe()

ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).